# 3_1 index text
* index `jinaai/JinaVDR`-`github-readme-retrieval-multilingual` dataset
* only index 'text' as 'text'

In [1]:
import os
import sys
parent_dir = os.path.dirname(os.path.dirname(os.getcwd()))
print(parent_dir)
core_src_dir = os.path.join(parent_dir, "src/psiking-core")
print(core_src_dir)
sys.path.append(core_src_dir)

from datasets import load_dataset, load_from_disk

from config import settings

/Users/id4thomas/github/psi-king
/Users/id4thomas/github/psi-king/src/psiking-core


# 1. Load Dataset
* `ko` split of `github-readme-retrieval-multilingual` 

In [2]:
ds = load_from_disk(
    os.path.join(
        settings.data_dir,
        'github-readme-retrieval-multilingual/data/ko'
    )
)

In [3]:
id_column = list(range(len(ds['test'])))
ds['test']=ds['test'].add_column(
    name='id',
    column=id_column
)

In [4]:
ds['test'][0]

{'query': '페이퍼 머니를 사용하는 주식 거래 앱입니다.',
 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=618x1080>,
 'image_filename': 'images/1249_screenshot.png',
 'license_type': 'APACHE_2',
 'license_text': '                                 Apache License\n                           Version 2.0, January 2004\n                        http://www.apache.org/licenses/\n\n   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION\n\n   1. Definitions.\n\n      "License" shall mean the terms and conditions for use, reproduction,\n      and distribution as defined by Sections 1 through 9 of this document.\n\n      "Licensor" shall mean the copyright owner or entity authorized by\n      the copyright owner that is granting the License.\n\n      "Legal Entity" shall mean the union of the acting entity and all\n      other entities that control, are controlled by, or are under common\n      control with that entity. For the purposes of this definition,\n      "control" means (i) the power, 

# 2. Prepare PSIKing Documents
* dataset text -> PSIKing Document

In [5]:
from psiking.core.base.schema import TextNode, Document

In [6]:
def prepare_documents(x):
    node = TextNode(
        text=x['text_description'],
        metadata={
            'docid': x['id']
        }
    )
    document = Document(
        nodes=[node],
        metadata={
            'docid': x['id']
        }
    )
    return {'doc': document.to_dict()}

In [7]:
collection = ds.map(
    function=prepare_documents,
    batched=False
)

In [8]:
collection['test'][6]

{'query': 'Bulma를 기반으로 하는 Vue.js용 경량 UI 구성 요소',
 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=540x1080>,
 'image_filename': 'images/774_screenshot.png',
 'license_type': 'MIT',
 'license_text': 'MIT License\n\nCopyright (c) 2017-2019 Rafael Beraldo\n\nPermission is hereby granted, free of charge, to any person obtaining a copy\nof this software and associated documentation files (the "Software"), to deal\nin the Software without restriction, including without limitation the rights\nto use, copy, modify, merge, publish, distribute, sublicense, and/or sell\ncopies of the Software, and to permit persons to whom the Software is\nfurnished to do so, subject to the following conditions:\n\nThe above copyright notice and this permission notice shall be included in all\ncopies or substantial portions of the Software.\n\nTHE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR\nIMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,\nFITNESS F

# 3. Chunk, Embed

## 3-1. Chunk Texts

In [9]:
from psiking.core.splitter.text.langchain_text_splitters import LangchainRecursiveCharacterTextSplitter

splitter = LangchainRecursiveCharacterTextSplitter(
    chunk_size = 1024,
    chunk_overlap = 128
)

In [10]:
chunks = []
for sample in collection['test']:
    document = Document.from_dict(sample['doc'])
    
    document_chunks = []
    source_id = document.id_
    for i, node in enumerate(document.nodes):
        # Run Splitter
        if isinstance(node, TextNode):
            try:
                split_nodes = splitter.run(node)
            except Exception as e:
                print(i, node)
                print(str(e))
                raise e
        else:
            split_nodes = [node]
        
        # Create New Document
        # print(len(document.nodes[0].text), "SPLIT:", len(split_nodes))
        for split_node in split_nodes:
            # Each Document contains single node
            chunk = Document(
                nodes=[split_node],
                metadata={
                    "source_id": source_id,
                    "docid": document.metadata['docid'],
                }
            )
            document_chunks.append(chunk)
    chunks.extend(document_chunks)
print(len(chunks))

1435


In [ ]:
# # Save Chunks (for testing)
# import json
# with open('cache/text/chunks.json', 'w') as f:
#     f.write(
#         json.dumps(
#             {
#               'chunks': [
#                   x.to_dict() for x in chunks
#               ]  
#             },
#             indent=4,
#             ensure_ascii=False
#         )
#     )

## 3-2. Embed with jina-emb-v4

In [11]:
import asyncio
from tqdm.asyncio import tqdm

from psiking.core.embedder.vllm.online_jina_emb_v4 import VLLMOnlineJinaEmbV4Embedder

In [12]:
VLLM_ENDPOINT="http://localhost:8080"
MODEL="jina-embeddings-v4-vllm-retrieval"
embedder = VLLMOnlineJinaEmbV4Embedder(
    base_url=VLLM_ENDPOINT,
    model=MODEL
)

In [13]:
async def embed(semaphore, doc: Document):
    text = doc.nodes[0].text
    async with semaphore:
        embedding = await embedder.arun(
            input=text,
            input_format='text',
            mode='passage',
            pool=True,
            normalize=True
        )
    return embedding

In [14]:
semaphore = asyncio.Semaphore(32)

tasks = []
for chunk in chunks:
    task = embed(semaphore, chunk)
    tasks.append(task)

embeddings = await tqdm.gather(*tasks)

100%|██████████| 1435/1435 [45:28<00:00,  1.90s/it] 


In [15]:
print(len(embeddings))
print(len(embeddings[0]))

1435
2048


In [22]:
type(embeddings[0].tolist()[0])

float

# 4. Insert into VectorStore

In [17]:
from qdrant_client import QdrantClient
from psiking.core.storage.vectorstore.qdrant import QdrantSingleVectorStore

# initialize client
client = QdrantClient(host="localhost", port=6333)
collection_name = "jinavdr-github-text"

vector_store = QdrantSingleVectorStore(
    collection_name=collection_name,
    client=client
)

In [18]:
from qdrant_client.http import models

embedding_dim = 2048

vector_store.create_collection(
    on_disk_payload=True,  # store the payload on disk
    vectors_config = models.VectorParams(
        size=embedding_dim,
        distance=models.Distance.COSINE,
        on_disk=True,
    )
)

In [23]:
# np array -> List[float]
embeddings_float = [
    x.tolist() for x in embeddings
]

In [24]:
vector_store.add(
    documents=chunks,
    embeddings=embeddings_float,
    metadata_keys=["source_file", "docid"]
)

In [25]:
chunks[0].id_

'd252b0ed-6859-4bc5-8191-f3b4b45c0ab5'

In [26]:
points = vector_store._client.retrieve(
    collection_name=vector_store.collection_name,
    ids=[chunks[0].id_],
    with_vectors=True
)

In [27]:
print(points[0].id)
print(points[0].payload)
print(len(points[0].vector))

d252b0ed-6859-4bc5-8191-f3b4b45c0ab5
{'docid': 0}
2048


In [28]:
points[0]

Record(id='d252b0ed-6859-4bc5-8191-f3b4b45c0ab5', payload={'docid': 0}, vector=[-0.0033416748, -0.025276402, 0.008666208, -0.018202744, 0.0043505304, 0.0005320401, 0.013703917, 0.0293541, -0.0068651093, 0.00443986, -0.011787904, 0.011765875, -0.010888652, 0.020539265, -0.033876643, -0.020044846, 0.020923723, -0.019098673, 0.055290785, -0.042829305, -0.031764764, 0.0067238132, 0.030262232, -0.0044128764, 0.0027097939, -0.02296366, -0.03833499, -0.011496338, 0.04241281, -0.020197786, 0.016363505, 0.020589076, 0.010935601, 0.007995582, -0.069539666, -0.009606065, -0.023414377, -0.009976561, -0.011715713, 0.0017687511, -0.0011005063, -0.01297089, -0.033130236, -0.05971473, -0.018460084, -0.0873191, -0.011168176, -0.014299016, 0.015436397, -0.020319104, 0.01678558, -0.0013228914, -0.045614373, -0.024760526, -0.0061385767, 0.02664643, 0.021925893, -0.0063431985, -0.0063191988, 0.0032560632, 0.0018315763, -0.026790673, 0.024502903, 0.013323255, 0.035172343, -0.03181186, -0.032946408, 0.022067